In [33]:
import pandas as pd

In [61]:
az_2021 = pd.read_excel('az_2021.xlsx')

In [34]:
janssen_2021 = pd.read_excel('janssen_2021.xlsx')

In [35]:
moderna_2021 = pd.read_excel('moderna_2021.xlsx')

In [27]:
pfizer_2021 = pd.read_excel('pfizer_2021.xlsx')

In [36]:
pfizer_2020 = pd.read_excel('pfizer_2020.xlsx')

In [29]:
all_2020_2021 = pd.concat([az_2021, janssen_2021, moderna_2021, pfizer_2020, pfizer_2021])

In [36]:
columns = [
    'EU Local Number', 'EV Gateway Receipt Date',
   'Primary Source Qualification',
   'Primary Source Country for Regulatory Purposes',
   'Patient Age Group',
   'Parent Child Report',
   'Patient Sex',
   'Reaction List PT (Duration – Outcome - Seriousness Criteria)',
   'Suspect/interacting Drug List (Drug Char - Indication PT - Action taken - [Duration - Dose - Route])',
   'Concomitant/Not Administered Drug List (Drug Char - Indication PT - Action taken - [Duration - Dose - Route])'
]

In [38]:
all_2020_2021[columns].to_csv('eudra_vigilance_2021_06_05.csv', sep='\t')

In [1]:
import pandas as pd
all_2020_2021 = pd.read_csv('eudra_vigilance_2021_06_05.csv', sep='\t')#.sample(10000)

In [2]:
all_2020_2021['reaction_list'] = all_2020_2021['Reaction List PT (Duration – Outcome - Seriousness Criteria)'].map(
    lambda x: x.split('<BR><BR>'))

In [3]:
def get_vaccine(_string):
    _dict = {'TOZINAMERAN': 'pfizer',
             'COMIRNATY': 'pfizer',
             'ASTRAZENECA': 'az',
            'CHADOX1': 'az', 'MODERNA': 'moderna', 'JANSSEN': 'janssen', 'AD26.COV2.S': 'janssen'}
    for key, value in _dict.items():
        if key in _string:
            return value
    return None

In [4]:
all_2020_2021['vaccine'] = all_2020_2021['Suspect/interacting Drug List (Drug Char - Indication PT - Action taken - [Duration - Dose - Route])'].map(
    get_vaccine)

In [5]:
all_2020_2021 = all_2020_2021[[
    'EU Local Number', 'EV Gateway Receipt Date',
    'Primary Source Qualification',
    'Primary Source Country for Regulatory Purposes', 'Patient Age Group',
    'Patient Sex',
    'reaction_list', 'vaccine']]

In [6]:
all_2020_2021.columns = [
    'EU Local Number', 'report_date', 'is_primary_source_professional', 'is_eee', 'age_group', 'sex',
    'reaction_list', 'vaccine'
]

In [7]:
# Performing a Flatmap Operation on a selected column
# containing rows of lists this example uses
# Pythons itertools
import pandas as pd
from itertools import chain


def flatmap(f, items):
        return list(chain.from_iterable(list(map(f, items))))

    
def create_dict(df, headers_ls, f_var, x):
    temp_dict = dict({})
    for h in headers_ls:
        temp_dict[h] = df[h]
    temp_dict[f_var] = x
    return temp_dict
    
    
def flatmap_df(df, flatten_col):
    headers_ls = df.columns
    rec_dict = df[headers_ls].to_dict(orient='records')
    return pd.DataFrame(flatmap(lambda rec_dict: [create_dict(rec_dict, 
                                        headers_ls, flatten_col, 
                                        col_ls) for col_ls in rec_dict[flatten_col]], rec_dict))

In [8]:
eudra_vigilance = flatmap_df(all_2020_2021, 'reaction_list')

In [9]:
import re
prog = re.compile(r'(.+)\(([\d\w\s\/,\.]+)\-\ ([\d\w\s\/,]+)\ \-([\d\w\s\/,]+)\)')
def parse_reaction_list(reaction_list):
    result = prog.match(reaction_list)
    if result:
        return {'reaction': result.group(1).strip(),
                'duration': result.group(2).strip(),
                'outcome': result.group(3).strip(),
                'seriousness': result.group(4).strip()}
    else:
        return {'reaction': None,
                'duration': None,
                'outcome': None,
                'seriousness': None}

In [10]:
eudra_vigilance[['reaction', 'duration', 'outcome', 'seriousness']] = pd.DataFrame(eudra_vigilance['reaction_list'].apply(parse_reaction_list).to_list())

In [11]:
eudra_vigilance['is_primary_source_professional'] = eudra_vigilance['is_primary_source_professional'].map(
    {'Non Healthcare Professional': 0, 'Healthcare Professional': 1}
)

In [12]:
eudra_vigilance['is_eee'] = eudra_vigilance['is_eee'].map(
    {'European Economic Area': 1, 'Non European Economic Area': 0}
).fillna(-1)

In [13]:
eudra_vigilance['sex'] = eudra_vigilance['sex'].map(
    {'Female': 2, 'Male': 1}
).fillna(0)

In [14]:
eudra_vigilance['age_group'] = eudra_vigilance['age_group'].map(
    {'18-64 Years': '18-64y',
     'More than 85 Years': '85+y',
     'Not Specified': '',
     '65-85 Years': '65-85y',
     '2 Months - 2 Years': '02m-2y',
     '12-17 Years': '12-17y',
     '3-11 Years': '3-11y',
     '0-1 Month': '00m-01m'}
).fillna(0)

In [15]:
eudra_vigilance.drop(columns=['reaction_list'], inplace=True)

In [16]:
eudra_vigilance['EU Local Number'] = eudra_vigilance['EU Local Number'].map(lambda x: x[6:]).astype(int)

In [17]:
flatmap_df(pd.DataFrame(eudra_vigilance.seriousness.map(lambda x: x.split(','))), 'seriousness').seriousness.map(
    lambda x: x.strip()).value_counts()

                                       1082748
Other Medically Important Condition     549811
Caused/Prolonged Hospitalisation        114094
Disabling                                91656
Life Threatening                         36041
Results in Death                         18368
Congenital Anomaly                         399
Name: seriousness, dtype: int64

In [18]:
_dict = {
    'medically_important': 'Other Medically Important Condition',
    'prolonged_hospit': 'Caused/Prolonged Hospitalisation',
    'disabling': 'Disabling',
    'life_threat': 'Life Threatening',
    'fatal': 'Results in Death'
}
for key, value in _dict.items():
    eudra_vigilance[key] = eudra_vigilance['seriousness'].map(lambda x:
                                                             1 if value in x else 0)

In [20]:
eudra_vigilance.drop(columns=['seriousness'], inplace=True)

In [21]:
eudra_vigilance.to_csv('eudra_vigilance_2021_06_05_clean.csv', sep='\t', index=False)

In [22]:
import pandas as pd
eudra_vigilance = pd.read_csv('eudra_vigilance_2021_06_05_clean.csv', sep='\t')

In [60]:
len(eudra_vigilance.query('fatal==1 or outcome=="Fatal"')['EU Local Number'].unique())

8175